In [48]:
require('nn')
require('hdf5')
require('optim')

f = hdf5.open("data.hdf5", "r")
X_train = f:read("train_input"):all()
Y_train = f:read("train_output"):all()
X_valid = f:read("valid_input"):all()
q_valid = f:read("valid_q"):all()
X_test = f:read("test_input"):all()
q_test = f:read("test_q"):all()
kaggle_valid = f:read("valid_kaggle"):all()
nclasses = f:read('nclasses'):all():long()[1]
nfeatures = f:read('nfeatures'):all():long()[1]
n_reduced = 1001
f:close()

window_size = X_train:size(2)
n_actual = nfeatures

In [49]:
n_actual


10002	


In [50]:

embedded_size = 30
D_h = 100

lookup1 = nn.LookupTable(nfeatures,embedded_size)
linear1 = nn.Linear(window_size*embedded_size, D_h)
tanh = nn.HardTanh()
linear2 = nn.Linear(D_h, n_actual)
h = nn.Sequential()
h:add(lookup1)
h:add(nn.Reshape(window_size*embedded_size, True))
h:add(linear1)
h:add(tanh)
h:add(linear2)

mse = nn.CrossEntropyCriterion()

In [ ]:
w, dl_dw = h:getParameters()


feval = function(w_new)
   
    bsize= batch
    local idx = torch.randperm(X_train:size(1)):sub(1,bsize)

    x = torch.Tensor(bsize, X_train:size(2))

    
    --y = y:reshape(Y_train:size(1), 1)
    
    y_ = torch.Tensor(bsize, 1)

    for i=1,bsize do
        x[i] = X_train[idx[i]]
        y_[i] = Y_train[idx[i]]
    end
    
    y_ = y_:squeeze()
    
    local inputs = x
    local targets = y_
    -- reset gradients (gradients are always accumulated, to accommodate
    -- batch methods)
    dl_dw:zero()

    -- evaluate the loss function and its derivative with respect to x, given a mini batch
    local prediction = h:forward(inputs)
    local loss_w = mse:forward(prediction, targets)
    h:backward(inputs, mse:backward(prediction, targets))
    return loss_w, dl_dw
            
end

batch = 32

-- cycle on data
for i = 1,1000 do
    -- train a mini_batch of batchSize in parallel
    _, fs = optim.adamax(feval,w)

    if i % 10 == 0 then
        print('loss for iteration ' .. i  .. ' is ' .. fs[1] )
        -- print(sgd_params)
    end
end

loss for iteration 10 is 4.3968732573569	


loss for iteration 20 is 5.2851650503275	


loss for iteration 30 is 5.0630694911214	


loss for iteration 40 is 4.6755060549749	


loss for iteration 50 is 4.574541745319	


...

loss for iteration 230 is 4.7205208810349	


loss for iteration 240 is 4.1348408492831	


loss for iteration 250 is 5.0310679510545	


loss for iteration 260 is 5.4126491739487	


loss for iteration 270 is 5.5975666421937	


In [107]:
sum = 0
for i = 1,X_valid:size(1) do
    a = h:forward(X_valid[i])
    b = nn.Index(1):forward{a,q_valid[i]}
    distribution = nn.LogSoftMax():forward(b)
    answer = distribution[kaggle_valid[i]]
    sum = sum + answer
end

sum = torch.exp(-sum/X_valid:size(1))

In [108]:
sum

6.8822395054492	


In [59]:
nn.Index(1):forward{a,q_valid[1]}

-5.1122
-5.5452
-5.7715
-5.7438
-5.5565
-5.7470
-5.5318
-6.0734
-5.5848
-5.4496
-5.2689
-6.0985
-5.5387
-5.6593
-5.8593
-5.7449
-5.7572
-5.9486
-5.9687
-5.1567
-6.1176
-5.6929
-5.8220
-5.9375
-5.3624
-5.9788
-5.6616
-5.3910
-5.5089
-5.7142
-5.4658
-5.4007
-5.6064
-6.2314
-5.5416
-5.6517
-5.4876
-5.8214
-5.7023
-5.7191
-3.0215
-2.8163
-5.2906
-6.1233
-5.4841
-5.2421
-6.0447
-5.7865
-5.3768
-5.6497
[torch.DoubleTensor of size 50]



In [57]:
torch.LongTensor(q_valid[1])

  407
 6937
 1655
  224
 1573
 6613
 8345
 8618
  297
 9494
 9085
 2261
 7107
 5195
 8009
 5554
 6273
  922
 7771
 5130
 2408
 9824
  660
 8177
 5070
 9016
 5779
 1460
 4601
 1555
 4583
 8021
 5839
 5433
 2645
 5701
 5699
 8434
 4013
 7236
    5
    6
 7391
 3700
 8474
 2839
 7576
  844
 6710
 4194
[torch.LongTensor of size 50]



In [ ]:
q_valid